In [74]:
import os

import numpy as np
import pandas as pd

from glob import glob
from math import sqrt
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import mean_squared_error, f1_score, classification_report
from sklearn.model_selection import train_test_split

In [75]:
home = os.path.dirname(os.getcwd())
data_path = os.path.join(home, "data")

In [76]:
dfs = {
    os.path.basename(file_name).split(".")[0]: 
    pd.read_csv(file_name) 
    for file_name in glob(f'{data_path}/*.csv')
}
for k, v in dfs.items():
    print(k)
    locals()[k] = v

problem_data
sample_submissions
test_submissions
train_submissions
user_data


In [77]:
user_data["user_id"] = user_data["user_id"].apply(lambda x: int(x.split('_')[1]))
user_data.sort_values("user_id", inplace=True)

In [78]:
problem_data["problem_id"] = problem_data["problem_id"].apply(lambda x: int(x.split('_')[1]))
problem_data.sort_values("problem_id", inplace=True)

In [79]:
train_submissions["problem_id"] = train_submissions["problem_id"].apply(lambda x: int(x.split('_')[1]))
train_submissions["user_id"] = train_submissions["user_id"].apply(lambda x: int(x.split('_')[1]))

test_submissions["problem_id"] = test_submissions["problem_id"].apply(lambda x: int(x.split('_')[1]))
test_submissions["user_id"] = test_submissions["user_id"].apply(lambda x: int(x.split('_')[1]))

In [80]:
train_data, valid_data = train_test_split(train_submissions, stratify=train_submissions["attempts_range"], test_size=0.2)

In [81]:
n_users = len(user_data)
n_problems = len(problem_data)

train_data_matrix = np.zeros((n_users, n_problems))
for line in train_data.to_dict(orient="records"):
    train_data_matrix[line['user_id']-1, line["problem_id"]-1] = line["attempts_range"]
    
valid_data_matrix = np.zeros((n_users, n_problems))
for line in valid_data.to_dict(orient="records"):
    valid_data_matrix[line['user_id']-1, line["problem_id"]-1] = line["attempts_range"]

In [82]:
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')
problem_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')

In [83]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #You use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

In [84]:
problem_prediction = predict(valid_data_matrix, problem_similarity, type='item')
user_prediction = predict(valid_data_matrix, user_similarity, type='user')

In [85]:
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten()
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

def rounded_f1_score(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten()
    prediction = np.round(prediction)
    prediction[prediction > 6] = 6
    prediction[prediction < 1] = 1
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    print(classification_report(ground_truth, prediction))
    
    return f1_score(ground_truth, prediction, average="weighted")

In [86]:
print('User-based CF RMSE: ' + str(rmse(user_prediction, valid_data_matrix)))
print('Item-based CF RMSE: ' + str(rmse(problem_prediction, valid_data_matrix)))

print('User-based CF F1: ' + str(rounded_f1_score(user_prediction, valid_data_matrix)))
print('Item-based CF F1: ' + str(rounded_f1_score(problem_prediction, valid_data_matrix)))

User-based CF RMSE: 2.042632545073892
Item-based CF RMSE: 2.057570949299947


c:\programdata\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\programdata\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

        1.0       0.53      1.00      0.70     16561
        2.0       0.00      0.00      0.00      9464
        3.0       0.00      0.00      0.00      2829
        4.0       0.00      0.00      0.00      1100
        5.0       0.00      0.00      0.00       499
        6.0       0.00      0.00      0.00       606

avg / total       0.28      0.53      0.37     31059

User-based CF F1: 0.3708738820490663
             precision    recall  f1-score   support

        1.0       0.53      1.00      0.70     16561
        2.0       0.00      0.00      0.00      9464
        3.0       0.00      0.00      0.00      2829
        4.0       0.00      0.00      0.00      1100
        5.0       0.00      0.00      0.00       499
        6.0       0.00      0.00      0.00       606

avg / total       0.28      0.53      0.37     31059

Item-based CF F1: 0.3708738820490663


In [87]:
print(classification_report([1, 2, 3, 4, 5], [2, 2, 2, 2, 2]))

             precision    recall  f1-score   support

          1       0.00      0.00      0.00         1
          2       0.20      1.00      0.33         1
          3       0.00      0.00      0.00         1
          4       0.00      0.00      0.00         1
          5       0.00      0.00      0.00         1

avg / total       0.04      0.20      0.07         5



c:\programdata\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
